In [ ]:
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BertForSequenceClassification,RobertaForSequenceClassification,
)

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)

with torch.no_grad():
    model.classifier.weight.zero_()
    model.classifier.bias.zero_()

In [ ]:
isinstance(model, RobertaForSequenceClassification)

In [ ]:
# model.classifier.weight.shape

In [ ]:
def update_classification_head(model, num_labels):
    # self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    classifier = torch.nn.Linear(model.config.hidden_size, num_labels)
    # copy old weights
    with torch.no_grad():
        classifier.weight.index_copy_(
            0,
            torch.tensor(range(model.classifier.weight.size(0)), dtype=torch.long),
            model.classifier.weight,
        )
        classifier.bias.index_copy_(
            0,
            torch.tensor(range(model.classifier.bias.size(0)), dtype=torch.long),
            model.classifier.bias,
        )
    # replace classifier
    model.classifier = classifier
    return model

In [ ]:
update_classification_head(model, 3).classifier.weight.data